In [1]:
#import of the needed packages
#if not installed, pip install them
import os
from string import punctuation
import re
import nltk
#nltk.download()
from nltk.corpus import stopwords
import spacy
from nltk import Tree
import time

Please change the below directory to where you pulled files. 

In [2]:
# change directory if needed
os.chdir("D:/Dataset/")

In [3]:
#use tf-idf to get the 5 most representative words for each test file 
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer

#generate the corpus
path_here = r'Work/'
text = open(path_here + "wordsEnglish.txt", errors = 'ignore', encoding = 'utf-8').read()

#only use the first 1000 files for demonstration
corpus = text.split("\n")
#create a list for the name of the files
indexes = []
count = 0
corpus_test = []
while count < 1000:
    if len(corpus[count].split(" ")) >= 5:
        corpus_test.append(corpus[count])
        indexes.append(count)
    count += 1
files = os.listdir("DatasetEnglish/")
list_in = []
for index in indexes:
    list_in.append(files[index])
print(list_in[:5]) 

['1001571.txt', '1005614.txt', '1005617.txt', '1005672.txt', '1005673.txt']


In [4]:
#initialize of the tf-idf transformer
transformer = TfidfTransformer()
#initialize of the vectorizer of words
vectorizer = CountVectorizer()
#vectorize the corpus
vector = vectorizer.fit_transform(corpus_test)
#calculate the tf-idf for all words
tfidf = transformer.fit_transform(vector)
#get all the words in the corpus
words = vectorizer.get_feature_names()

In [5]:
#get the weights for each word in each file in the corpus
weights = tfidf.toarray()
#create a dictionary to store the five most representative words for each file
dict_all = {}

In [6]:
#go through all the files 
for i in range(0, len(weights)):
    weights_i = weights
    #print("---{}---".format(dict_in[i]))
    max_word_tfidf = {}
    for p in range(0, 5):
        #go through the file five times to find the five most representative words 
        max_word = ""
        max_tfidf = 0
        for j in range(len(words)):
            #if word already in the dictionary of representative words, continue, to find the second/third/... representative words 
            if words[j] in max_word_tfidf.values():
                continue
            #find the current most representative words(not considering the words already in the max_word_tfidf dictionary)
            if weights_i[i][j] > max_tfidf:
                max_word = words[j]
                max_tfidf = weights_i[i][j]
        #print(max_word)
        #print(max_tfidf)
        max_word_tfidf[p] = max_word
    dict_all[list_in[i]] = list(max_word_tfidf.values())
for k, v in list(dict_all.items()):
    if len(list(set(v))) < 5:
        del dict_all[k]
#print the final dictionary of the five most representative words
print({k: dict_all[k] for k in list(dict_all)[:10]})

{'1001571.txt': ['london', 'centre', 'york', 'respondent', 'city'], '1005614.txt': ['carlyle', 'asia', 'ii', 'firm', 'partners'], '1005617.txt': ['boj', 'rate', 'japan', 'monetary', 'yen'], '1005672.txt': ['avon', 'direct', 'sale', 'application', 'china'], '1005673.txt': ['annualise', 'tighten', 'growth', 'quarter', 'clearly'], '1005674.txt': ['leak', 'bp', 'malone', 'worker', 'shut'], '1005676.txt': ['gome', 'paradise', 'china', 'appliance', 'retailer'], '1005677.txt': ['paper', 'york', 'times', 'save', 'newsprint'], '1005678.txt': ['rate', 'government', 'growth', 'investment', 'effective'], '1005679.txt': ['rosneft', 'temasek', 'cnpc', 'stake', 'acquire']}


In [7]:
import pandas as pd
#change the dictionary to dataframe
df = pd.DataFrame(dict_all)
#transpose of dataframe
df = df.T
#add a filename column to the dataframe
df['filename'] = df.index
#print the dataframe
df

,0,1,2,3,4,filename
1001571.txt,london,centre,york,respondent,city,1001571.txt
1005614.txt,carlyle,asia,ii,firm,partners,1005614.txt
1005617.txt,boj,rate,japan,monetary,yen,1005617.txt
1005672.txt,avon,direct,sale,application,china,1005672.txt
1005673.txt,annualise,tighten,growth,quarter,clearly,1005673.txt
...,...,...,...,...,...,...
1007401.txt,arenas,masters,qizhong,racetrack,tournament,1007401.txt
1007402.txt,carlyle,xugong,deal,machinery,revise,1007402.txt
1007403.txt,socgen,bank,citigroup,foreign,consortium,1007403.txt
1007404.txt,surplus,foreign,hot,trade,speculative,1007404.txt


In [8]:
#for all files
if os.path.isfile("Work/CommunityEnglishTest.txt") == False:
    for i in range(len(df)):
        #for each file, get the 
        for j in range(len(df)):
            if i != j:
                list_all = []
                for p in range(0, 5):
                    list_all.append(df.iloc[i, p])
                    list_all.append(df.iloc[j, p])
                list_all = list(set(list_all))
                #print(list_all)
                duplicate = 10 - len(list_all)
                if duplicate > 0:
                    with open ("Work/CommunityEnglishTest.txt", 'a', \
                               encoding = 'utf-8') as f:
                        #a single line for each file 
                        text_write = str(df.index[i].split(".")[0]) + "\t" + str(df.index[j].split(".")[0]) + "\t" + str(duplicate) + "\n"
                        #print each line
                        print(text_write)
                        #write the words into the file
                        f.write(text_write)

In [9]:
if os.path.isfile("Work/TopFiveWords.csv") == False:
    df.to_csv('TopFiveWords.csv',encoding = "utf-8-sig", index = True)

In [10]:
#the function for loading the txt file
def loadtxt(path):
    #read the txt file
    with open(path, "r") as f:
        #create two dictionaries for to store the vectors and the edges
        dict_vec = {}
        dict_edge = {}
        #go through each line in the txt file
        for line in f.readlines():
            #for each node in each line
            for i in range(0, 2):
                #if node already in list, get its edges list
                if line.split("\t")[i] in dict_vec:
                    list_edges = dict_edge[line.split("\t")[i]]
                else:
                    #if node not in list, initialize its edges list
                    list_edges = []
                    #put the node in dict_vec
                    dict_vec[line.split("\t")[i]]  = True
                #append the edge to the edges list of the node
                list_edges.append(line.split("\t")[1 - i] + ":" \
                                 + line.split("\t")[2].rstrip())
                #update the edges list for the certain node
                dict_edge[line.split("\t")[i]] = list_edges
    #return the final dictionary of vectors and edges
    return dict_vec, dict_edge
#show the example output
#loadtxt("Community.txt")

In [11]:
#the function for calculating the modularity
def calculate_modularity(dict_vec, dict_edge):
    #initialize the modurality value
    q = float(0)
    #initialize the sum of all the weights
    sum_of_weights = float(0)
    #get the nodes (files) in the network
    nodes = dict_edge.keys()
    #define a dictionary for the communities with corresponding nodes
    dict_community = {}
    
    #go through all the nodes to find the total sum of weights
    for i in nodes:
        #get the list of edges for each node
        list_edges = dict_edge[i]
        #get the weight of each edge for each node 
        for j in range(0, len(list_edges)):
            #add the weight of the edge to the total sum of weights
            sum_of_weights = sum_of_weights + float(list_edges[j].split(":")[1])
        
        #put nodes in the community dictionary
        #get the community that the current node is in 
        community = dict_vec[i]
        if community in dict_community:
            list_community = dict_community[community]
        else:
            list_community = []
        #append the current node to the list of the current community
        list_community.append(i)
        #put the list of the current community to the dictionary of communities
        dict_community[community] = list_community
    
    #get the communities
    communities = dict_community.keys()
    #go through all the communities
    for i in communities:
        #initialize the sum within community
        insum = float(0)
        #initialize the sum of the weight of the edges related to the community
        tolsum = float(0)
        #get the list of nodes from the same community
        list_vec = dict_community[i]
        for j in range(0, int(len(list_vec))):
            dict_temp = {}
            node_j = list_vec[j]
            edges_node_j = dict_edge[node_j]
            #update the dictionary
            for edge in edges_node_j:
                dict_temp[edge.split(":")[0]] = edge.split(":")[1]
            for k in range(int(len(list_vec))):
                if list_vec[k] in dict_temp:
                    weight = dict_temp[list_vec[k]] 
                    insum = insum + float(weight)
                    
        #calculate the total sum 
        for node in list_vec:
            for i in dict_edge[node]:
                tolsum = tolsum + float(i.split(":")[1])
        #calculate the addition to the modularity
        temp_1 = insum / sum_of_weights
        temp_2 = tolsum / sum_of_weights
        q = q + temp_1 - (temp_2 ** 2)
    
    #return the final modularity
    return q

In [12]:
#define a function to change the community
def change_community(dict_vec, dict_edge, q):
    #initialize the Q
    Q = q
    #initialize the initial community
    dict_community = {}
    for node in dict_vec:
        #print(node)
        dict_community[node] = dict_vec[node]
        
    #go through different kinds of ways to change the community
    for node in dict_community:
        #print(node)
        #define a list of neighbors for each node
        list_neighbors = dict_edge[node]
        for edge in list_neighbors:
            community_initial = dict_community[node]
            vector = edge.split(":")[0]
            if community_initial != dict_community[vector]:
                dict_community[node] = dict_community[vector]
                if (calculate_modularity(dict_community, dict_edge) - Q) <= 0:
                    dict_community[node] = community_initial
                else:
                    Q = calculate_modularity(dict_community, dict_edge)
    return dict_community, Q

In [13]:
#define a function to calculate the number of communities
#also update the name of the communities 
def modify_community(dict_vec):
    #set a number for the total number of communities
    number_of_community = 0
    #get the current communities and the vectors
    communities = dict_vec.values()
    vectors = dict_vec.keys()
    #initialize a dictionary of communities
    dict_community = {}
    for name_community in communities:
        if name_community not in dict_community:
            dict_community[name_community] = number_of_community
            number_of_community = number_of_community + 1
    for vector in vectors:
        #get the community for the certain vector
        community_for_vector = dict_vec[vector]
        #update the community for vector
        dict_vec[vector] = dict_community[community_for_vector]
    #return the number of communities
    return number_of_community

In [14]:
#define a function to rebuild the graph
def rebuild_graph(vector_dict, edge_dict, community_num):
    vector_new_dict = {}
    edge_new_dict = {}
    community_dict = {}
    vectors = vector_dict.keys()
    communities = community_dict.keys()
    for key in vectors:
        if vector_dict[key] in community_dict:
            community_list = community_dict[vector_dict[key]]
        else:
            community_list = []
        community_list.append(key)
        community_dict[vector_dict[key]] = community_list
    for key in community_dict.keys():
        vector = str(key)
        vector_new_dict[vector] = vector
    for i in community_dict.keys():
        sum_in = float(0)
        vector_list = community_dict[i]
        if '0' in vector_list:
            print(vector_list)
            print(i)
        for j in range(len(vector_list)):
            link_list = edge_dict[vector_list[j]]
            tmp_dict = {}
            for link_mem in link_list: 
                tmp_dict[link_mem.split(":")[0]] = link_mem.split(":")[1]
            for k in range(len(vector_list)):
                if vector_list[k] in tmp_dict:
                    sum_in = sum_in + float(tmp_dict[vector_list[k]])
        inner_list = []
        inner_list.append(str(i) + ":" + str(sum_in))
        edge_new_dict[str(i)] = inner_list
    community_list = list(community_dict.keys())
    for i in range(0, len(community_list)):
        for j in range(0, len(community_list)):
            if i != j:
                sum_outer = float(0)
                member_list_1 = community_dict[community_list[i]]
                member_list_2 = community_dict[community_list[j]]
                for i_1 in range(0, len(member_list_1)):
                    tmp_dict = {}
                    tmp_list = edge_dict[member_list_1[i_1]]
                    for k in range(0, int(len(tmp_list))):
                        tmp = tmp_list[k].strip().split(":")
                        tmp_dict[tmp[0]] = tmp[1]
                    for j_1 in range(0, int(len(member_list_2))):
                        if member_list_2[j_1] in tmp_dict:
                            sum_outer = sum_outer + float(tmp_dict[member_list_2[j_1]])
                if sum_outer != 0:
                    inner_list = edge_new_dict[str(community_list[i])]
                    inner_list.append(str(j) + ":" + str(sum_outer))
                    edge_new_dict[str(community_list[i])] = inner_list
    return vector_new_dict, edge_new_dict, community_dict

def fast_unfolding(vector_dict, edge_dict):
    vectors = vector_dict.keys()
    for i in vectors:
        vector_dict[i] = i
    Q = calculate_modularity(vector_dict, edge_dict)
    Q_new = float(0)
    while (Q_new != Q):
        Q_new = Q
        vector_dict, Q = change_community(vector_dict, edge_dict, Q)
    print("start")
    community_num = modify_community(vector_dict)
    print("Q = ", Q)
    print("community number: ", community_num)
    Q_best = Q
    iteration = 1
    while True:
        print("\nbuild " + str(iteration))
        iteration += 1
        vector_new_dict, edge_new_dict, community_dict = rebuild_graph(vector_dict, edge_dict, community_num)
        Q_new = float(0)
        while (Q_new != Q):
            Q_new = Q
            vector_new_dict, Q = change_community(vector_new_dict, edge_new_dict, Q)
        community_num = modify_community(vector_new_dict)
        print("modularity: ", Q)
        print("community number: ", community_num)
        print("community dictionary: ", community_dict)
        if (Q_best == Q):
            break
        Q_best = Q
        vector_result = {}
        communities = community_dict.keys()
        for key in communities:
            value_of_vector = community_dict[key]
            for i in range(len(value_of_vector)):
                vector_result[value_of_vector[i]] = str(vector_new_dict[str(key)])
        for key in vector_result.keys():
            vector_dict[key] = vector_result[key]
    vector_result = {}
    for key in community_dict.keys():
        value_of_vector = community_dict[key]
        for i in range(len(value_of_vector)):
            vector_result[value_of_vector[i]] = str(vector_new_dict[str(key)])
    for key in vector_result.keys():
        vector_dict[key] = vector_result[key]
    print("\nbest modularity: " + str(Q_best))
    print("final community number: " + str(community_num))
    print("final community dictionary: ", community_dict)

In [15]:
vector_dict, edge_dict = loadtxt(r"Work\\CommunityEnglishTest.txt")
#vector_dict, edge_dict
fast_unfolding(vector_dict, edge_dict)

start
Q =  0.2494447569857315
community number:  109

build 1
modularity:  0.5862868887402405
community number:  26
community dictionary:  {0: ['1001571', '1005677', '1005716', '1006120', '1006128', '1006251', '1006262', '1006296', '1006337', '1006469', '1006724', '1006808', '1007294', '1007332', '1006545'], 1: ['1005920', '1006761', '1007291', '1005672', '1005676', '1005782', '1005795', '1005809', '1005815', '1005927', '1005948', '1006002', '1006049', '1006171', '1006263', '1006301', '1006326', '1006328', '1006333', '1006509', '1006550', '1006573', '1006628', '1006645', '1006674', '1006763', '1006791', '1006825', '1006843', '1006895', '1006953', '1006959', '1007092', '1007187', '1007239', '1007274', '1005875', '1006306', '1006847', '1007126', '1005954', '1006468', '1006960', '1007173', '1006834'], 2: ['1006751', '1006845', '1007059', '1007194', '1007095', '1007167', '1006045', '1006603', '1005775', '1005957', '1006651', '1006902', '1006928', '1005998', '1007186', '1006659', '1007018']